In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline
# This blokc is important if we want the memory to grow on the GPU, and not block allocate the whole thing
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from datetime import datetime
import os
from pathlib import Path
from sklearn.utils import class_weight 
# Hyperparam opt
import talos as ta

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

# Set path to find modelling tools for later use
import sys
sys.path.append(os.path.join(os.getcwd(),".."))
# Global params live here
import haberrspd.charCNN.globals

Using TensorFlow backend.


## Assign a loss function

In [ ]:
loss_func = 'squared_hinge'

if loss_func == 'hinge' or loss_func == 'squared_hinge':
    y_train = [-1 if x==0 else x for x in y_train]
    y_test = [-1 if x==0 else x for x in y_test]
    
if loss_func == 'binary_crossentropy':
    # Check if label-space is correct
    if (-1 in y_train) or (-1 in y_test):
        y_train = [0 if x==-1 else x for x in y_train]
        y_test = [0 if x==-1 else x for x in y_test]

## Train model

In [ ]:
# Compile
model.compile(loss=loss_func,  # TODO: change to cosine loss, cosine_proximity, binary_crossentropy
              optimizer='adam',            # TODO: check which is most appropriate
              metrics=['accuracy'])        # Probs other options here which are more useful

# Check if checkpoints dir exists, if not make it
if not os.path.exists('../../keras_checkpoints'):
    os.makedirs('../../keras_checkpoints')

# Callbacks
file_name = "char-CNN"
check_cb = ModelCheckpoint(file_name + '.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=0,
                           save_best_only=True,
                           mode='min')

earlystop_cb = EarlyStopping(monitor='val_loss',
                             patience=7,
                             verbose=0,
                             mode='auto')

# history = LossHistory()
"""
TODO:

-Add class-weight option to take into account class-imbalance on patients and controls
"""
fit_hist = model.fit(X_train,
                     y_train,
                     validation_data=(X_test, y_test),
                     verbose=0, # Set to zero if using live plotting of losses
                     class_weight = class_weights,
                     batch_size=128,
                     epochs=40,
                     #shuffle=True, # Our data is already shuffled during data loading
                     callbacks=[
                                #check_cb,
                                #tensorboard_callback,
                                PlotLossesCallback(),
                                #earlystop_cb
                               ]
                    )

# TALOS: hyperparameter optimisation

In [3]:
from haberrspd.charCNN.models_tf import char_cnn_model_talos
from haberrspd.charCNN.auxiliary_tf import create_training_data_keras
from numpy import vstack, asarray

### Main

In [25]:
class_weights,max_sentence_length = None,None # These needs to be set
# Set the parameter space
opt_params ={'conv_output_space' : [8,16,32,64],
             'number_of_large_filters' : [1,2,3,4],             
             'number_of_small_filters' : [1,2,3,4],
             'large_filter_lengt' : [30,60,120],
             'small_filter_lengt' : [5,10,20],
             'pool_length' : [2,4,8,16],
             'dense_units_layer_3' : [32,16,8,4],
             'dense_units_layer_2' : [32,16,8,4],
             'batch_size': [8,16,32,64],
             'epochs': [200], # Use early stopping instead
             'dropout': (0, 0.5, 5),
             'conv_kernel_initializer': ['uniform','normal'],
             'conv_bias_initializer': ['uniform','normal'],
             'dense_kernel_initializer': ['uniform','normal'],
             'dense_bias_initializer': ['uniform','normal'],
             'optimizer': ['adam', 'nadam', 'rmsprop'],
             'loss': ['logcosh', 'binary_crossentropy'],
             'conv_activation':['relu', 'elu'],
             'dense_activation':['relu', 'elu'],
             'last_activation': ['sigmoid'],
             # Stationary parameters, i.e. do not get optimised
             'max_sentence_length':[max_sentence_length]
            }

"""
'sgd': SGD,
'rmsprop': RMSprop,
'adagrad': Adagrad,
'adadelta': Adadelta,
'adam': Adam,
'adamax': Adamax,
'nadam': Nadam
"""

def size_of_optimisation_space(opt_params):
    space = 1
    for attribute in opt_params.keys():
        space*=len(opt_params[attribute])
        
    return space

int(size_of_optimisation_space(opt_params)*0.01)

382205952

### Small

In [4]:
from keras.optimizers import Adam, Nadam

DATA_ROOT = Path("../data/") / "MJFF" / "preproc" # Note the relative path
# Load training data and auxiliary variables
X_train, X_test, y_train, y_test, max_sentence_length,alphabet_size = \
create_training_data_keras(DATA_ROOT,"EnglishData-preprocessed.csv")

class_weights = dict(zip([0,1], 
                         class_weight.compute_class_weight('balanced',list(set(y_train)),y_train)))
print(class_weights)

params ={
         # Learning rate
         'lr': (0.1, 10, 3),
         'conv_output_space' : [4],#,8],
         'number_of_large_filters' : [2],             
         'number_of_small_filters' : [2],
         'large_filter_length' : [60],
         'small_filter_length' : [5],
         'pool_length' : [2],
         'dense_units_layer_3' : [32],
         'dense_units_layer_2' : [16],
         'batch_size': [32],
         'epochs': [100],
         'dropout': [0.05],#,0.1,0.2],
         'conv_kernel_initializer': ['uniform'],
         'conv_bias_initializer': ['uniform'],
         'dense_kernel_initializer': ['uniform'],
         'dense_bias_initializer': ['uniform'],
         'optimizer': [Adam],
         'loss': ['binary_crossentropy'],
         'conv_activation':['relu'],
         'dense_activation':['relu'],
         'last_activation': ['sigmoid'],
         # Stationary parameters, i.e. do not get optimised
         'max_sentence_length':[max_sentence_length],
         'alphabet_size':[alphabet_size],
         'control_class_weight' : [class_weights[0]],
         'pd_class_weight' : [class_weights[1]],
            }

Total number of characters: 49
{0: 0.7851796407185628, 1: 1.3766404199475066}


### Run Talos

In [5]:
t = ta.Scan(x=X_train,
            y=asarray(y_train).reshape(-1, 1),
            x_val=X_test, 
            y_val=asarray(y_test).reshape(-1, 1),
            model=char_cnn_model_talos,
            disable_progress_bar=False,
            params=params)
#             grid_downsample=0.01,  # Randomly samples 1% of the grid

  0%|          | 0/3 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 1049 samples, validate on 117 samples
Epoch 1/100
 - 5s - loss: 0.6932 - acc: 0.6368 - val_loss: 0.6904 - val_acc: 0.6154
Epoch 2/100
 - 3s - loss: 0.6932 - acc: 0.6368 - val_loss: 0.6905 - val_acc: 0.6154
Epoch 3/100
 - 3s - loss: 0.6932 - acc: 0.6368 - val_loss: 0.6906 - val_acc: 0.6154
Epoch 4/100
 - 3s - loss: 0.6932 - acc: 0.6368 - val_loss: 0.6906 - val_acc: 0.6154


 33%|███▎      | 1/3 [00:16<00:32, 16.17s/it]

Train on 1049 samples, validate on 117 samples
Epoch 1/100
 - 4s - loss: 0.6935 - acc: 0.3813 - val_loss: 0.6949 - val_acc: 0.3846
Epoch 2/100
 - 3s - loss: 0.6667 - acc: 0.5701 - val_loss: 0.6235 - val_acc: 0.7265
Epoch 3/100
 - 3s - loss: 0.6123 - acc: 0.6978 - val_loss: 0.6124 - val_acc: 0.7179
Epoch 4/100
 - 3s - loss: 0.5922 - acc: 0.7350 - val_loss: 0.6100 - val_acc: 0.7436
Epoch 5/100
 - 3s - loss: 0.6017 - acc: 0.6997 - val_loss: 0.5846 - val_acc: 0.7265
Epoch 6/100
 - 3s - loss: 0.5957 - acc: 0.7045 - val_loss: 0.6004 - val_acc: 0.7436
Epoch 7/100
 - 3s - loss: 0.5885 - acc: 0.6826 - val_loss: 0.5984 - val_acc: 0.7436
Epoch 8/100
 - 3s - loss: 0.5806 - acc: 0.7102 - val_loss: 0.6173 - val_acc: 0.6923


 67%|██████▋   | 2/3 [00:43<00:19, 19.63s/it]

Train on 1049 samples, validate on 117 samples
Epoch 1/100
 - 4s - loss: 0.6934 - acc: 0.3632 - val_loss: 0.6942 - val_acc: 0.3846
Epoch 2/100
 - 3s - loss: 0.6934 - acc: 0.3756 - val_loss: 0.6936 - val_acc: 0.3846
Epoch 3/100
 - 3s - loss: 0.6933 - acc: 0.4776 - val_loss: 0.6922 - val_acc: 0.6154
Epoch 4/100
 - 3s - loss: 0.6934 - acc: 0.4929 - val_loss: 0.6919 - val_acc: 0.6154
Epoch 5/100
 - 3s - loss: 0.6933 - acc: 0.5691 - val_loss: 0.6935 - val_acc: 0.3846
Epoch 6/100
 - 3s - loss: 0.6933 - acc: 0.4109 - val_loss: 0.6935 - val_acc: 0.3846
Epoch 7/100
 - 3s - loss: 0.6933 - acc: 0.4366 - val_loss: 0.6924 - val_acc: 0.6154


100%|██████████| 3/3 [01:08<00:00, 21.06s/it]


In [9]:
# Get weights passed to activation function
from talos import Deploy, Predict
import numpy as np

In [7]:
p = Predict(t)

In [10]:
p.predict(X_test[1][np.newaxis,:,:])

array([[0.43626744]], dtype=float32)

In [11]:
y_test[1]

1

In [12]:
Deploy(scan_object=t, model_name='test_deploy',metric='val_acc')

FileExistsError: [Errno 17] File exists: 'test_deploy'